In [1]:
!pip install ollama

In [2]:
import ollama
response = ollama.chat(model='mistral:7b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

 The sky appears blue due to a process called Rayleigh scattering. As sunlight travels from the sun to Earth, it consists of different wavelengths or colors of light. These colors are red (longest wavelength), orange, yellow, green, blue, indigo, and violet (shortest wavelength).

When sunlight reaches the Earth's atmosphere, it meets molecules and tiny particles suspended in the air, such as nitrogen and oxygen. These molecules scatter short-wavelength light (blue and violet) more than they do longer wavelengths (red, orange, yellow, indigo, and green). However, we see the sky as blue rather than violet because our eyes are more sensitive to blue light and because sunlight reaches us more from the blue part of the spectrum rather than violet. The blue light scatters in all directions, filling up the sky with a blue hue.

Additionally, some of the violet light gets absorbed by the ozone layer, further reducing the amount of violet light that reaches our eyes and making the sky appear m

In [3]:
prompt="[AVAILABLE_TOOLS] [{\"type\": \"function\", \"function\": {\"name\": \"get_current_weather\", \"description\": \"Get the current weather\", \"parameters\": {\"type\": \"object\", \"properties\": {\"location\": {\"type\": \"string\", \"description\": \"The city and state, e.g. San Francisco, CA\"}, \"format\": {\"type\": \"string\", \"enum\": [\"celsius\", \"fahrenheit\"], \"description\": \"The temperature unit to use. Infer this from the users location.\"}}, \"required\": [\"location\", \"format\"]}}}][/AVAILABLE_TOOLS] [INST] What is the weather like today in San Francisco [/INST]"
ollama.generate(model='mistral:7b', prompt=prompt, raw=True,stream=False)

{'model': 'mistral:7b',
 'created_at': '2024-05-23T21:35:31.43687004Z',
 'response': ' To find out, I need to call the `get_current_weather` function with the required parameters:\n\n[TOOL_CALLS] [{"name": "get_current_weather", "arguments": {"location": "San Francisco", "format": "celsius"}}]',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 18826163004,
 'load_duration': 1410164,
 'prompt_eval_count': 136,
 'prompt_eval_duration': 9359333000,
 'eval_count': 59,
 'eval_duration': 9423695000}

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

In [ ]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    }
]

In [ ]:
import functools

names_to_functions = {
  'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
  'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df)
}

In [ ]:
messages=[
  {
    'role': 'user',
    'content': "What's the status of my transaction T1001?",
  },
]

In [ ]:
response = ollama.chat(model='mistral:7b', messages=messages)


In [ ]:
response['model']

In [ ]:
messages.append({'role':"assistant", 'content':response['message']['content']})
messages.append({'role':"user", 'content':"My transaction ID is T1001."})